# Loan Data from Prosper


Prosper is a peer-to-peer lending platform, where borrowers with fair or good credit can apply for loans and investors can fund them. To qualify applicants, Prosper uses a proprietary rating system that considers data points like credit history and debt-to-income ratio. Then, a prospective borrower is assigned a Prosper score, which investors use to decide whether to fund the loan. 

When a loan is listed on the platform, it is open for funding by investors. Once a certain percentage of the loan amount has been funded by investors, the loan is considered "funded" and the borrower can access the funds. The loan application expires if the loan request isn’t at least 70% funded within 14 days, though most loans are funded within three days, according to the company. Source: [nerdwallet.com](https://www.nerdwallet.com/reviews/loans/personal-loans/prosper-personal-loans#:~:text=Then%2C%20a%20prospective%20borrower%20is,days%2C%20according%20to%20the%20company)



# Preliminary Data Wrangling

The loan dataset contains 113,937 loans with 81 variables on each loan, including loan amount, borrower rate (or interest rate), current loan status, borrower income, and many others. This [dictionary](https://docs.google.com/spreadsheets/d/1djKkeentYxKqgNFO2ZHF-KFwVunoZ83VbyLyWYz82M8/edit?usp=sharing) explains the variables (i.e columns) in the data set.

For this project, focus will be on just 10 - 20 of the variables in the dataset to ask and answer frequently asked questions like:  What factors affect a loan's outcome status? What affects the borrower's interest rate or annual percentage rate (APR)? How do loans differ based on the size of the loan amount? These are some of the questions that I will attempt to answer in this project.

Before any exploratory data analysis could be done, data wrangling/pre-processing was necessary to assess and clean up all quality and tidiness issues in the dataset


In [86]:
# import all packages and libraries to be used for data preprocessing and exploratory data analysis
import numpy as np
import pandas as pd
import boto3 #for connecting to AWS S3
import creds #creds is a python script containing the ACCESS KEY and SECRET KEY strings
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Loading the Gathered Data

Dataset was downloaded as a csv file from Udacity's Amazon AWS server where the csv file was hosted. The csv file will be loaded into a Pandas Dataframe prior to data assessment and data cleaning.

In [2]:
# Only used this step to save cost on AWS S3; to be removed after project completion
# Loading the already downloaded Dataset into a pandas dataframe, and previewing the dataframe
df = pd.read_csv('prosperLoanData.csv')

# Display the first 5 rows in the df
df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


In [3]:
# Connecting to AWS S3 using the AWS credentials

session = boto3.Session(
    aws_access_key_id=creds.ACCESS_KEY,
    aws_secret_access_key=creds.SECRET_KEY
)

s3 = session.client('s3')

In [9]:
# Loading the CSV file from the S3 bucket into a Pandas dataframe:

obj = s3.get_object(Bucket='prosper-loan', Key='prosperLoanData.csv')
df = pd.read_csv(obj['Body'])

df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


<br>

Now I have an idea of the values and types of records in the dataset, I also see that there are 81 columns in the dataset.

**Reminder: For an explanation of the different variables (i.e columns) in the dataset, check out this [dictionary](https://docs.google.com/spreadsheets/d/1djKkeentYxKqgNFO2ZHF-KFwVunoZ83VbyLyWYz82M8/edit?usp=sharing)**

## Assessing the Data

In this section, I would assess the data to look out for quality issues and tidiness issues, using both visual assessment and programmatic assessement.


### Visual Assessment

In [5]:
# Changing default settings to allow pandas dataframe to show more columns and expand the column width, since I want to visually assess the Dataframe.
# Since I know the dataset has 81 columns, I will set max_columns to 100 well above the 81 columns to ensure everything is displayed
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', 100)


<br>

There are 113,937 loans hence 113,937 rows in the dataframe. **It will not be efficient to visually inspect all 113,937 rows so I will instead display 20 samples from the dataset, and do a thorough assessment later during programmatic assessment.**

In [9]:
# Changing default row display setttings so that Pandas does not automatically collapse rows.
# Since I am requesting for 20 random rows, setting the max rows to 101 ensures that up to 100 rows can be displayed at once
pd.set_option('display.max_rows', 101)

# Now displaying 20 random rows in the dataframe for visual assessment
df.sample(20)

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,EstimatedEffectiveYield,EstimatedLoss,EstimatedReturn,ProsperRating (numeric),ProsperRating (Alpha),ProsperScore,ListingCategory (numeric),BorrowerState,Occupation,EmploymentStatus,EmploymentStatusDuration,IsBorrowerHomeowner,CurrentlyInGroup,GroupKey,DateCreditPulled,CreditScoreRangeLower,CreditScoreRangeUpper,FirstRecordedCreditLine,CurrentCreditLines,OpenCreditLines,TotalCreditLinespast7years,OpenRevolvingAccounts,OpenRevolvingMonthlyPayment,InquiriesLast6Months,TotalInquiries,CurrentDelinquencies,AmountDelinquent,DelinquenciesLast7Years,PublicRecordsLast10Years,PublicRecordsLast12Months,RevolvingCreditBalance,BankcardUtilization,AvailableBankcardCredit,TotalTrades,TradesNeverDelinquent (percentage),TradesOpenedLast6Months,DebtToIncomeRatio,IncomeRange,IncomeVerifiable,StatedMonthlyIncome,LoanKey,TotalProsperLoans,TotalProsperPaymentsBilled,OnTimeProsperPayments,ProsperPaymentsLessThanOneMonthLate,ProsperPaymentsOneMonthPlusLate,ProsperPrincipalBorrowed,ProsperPrincipalOutstanding,ScorexChangeAtTimeOfListing,LoanCurrentDaysDelinquent,LoanFirstDefaultedCycleNumber,LoanMonthsSinceOrigination,LoanNumber,LoanOriginalAmount,LoanOriginationDate,LoanOriginationQuarter,MemberKey,MonthlyLoanPayment,LP_CustomerPayments,LP_CustomerPrincipalPayments,LP_InterestandFees,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
107860,89E13538878932106170D84,555729,2012-02-02 06:32:57.293000000,NaN,36,Chargedoff,2012-07-07 00:00:00,0.29394,0.2552,0.2452,0.23920,0.1190,0.12020,3.0,D,6.0,1,NJ,Sales - Commission,Employed,16.0,False,False,NaN,2012-02-01 14:50:53,700.0,719.0,2002-08-21 00:00:00,2.0,2.0,26.0,1,16.0,2.0,5.0,0.0,0.0,10.0,2.0,0.0,0.0,0.00,21500.0,20.0,0.80,0.0,NaN,"$50,000-74,999",False,5000.000000,09993642102382516539F93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,733,5.0,25,60004,9500,2012-02-07 00:00:00,Q1 2012,EA6833649534182333D4FED,380.34,0.0000,0.00,0.0000,0.00,0.0,9500.00,9500.00,0.0,1.0,0,0,0.0,208
19354,D1FF3604255586102946AF4,1184323,2014-02-26 04:50:20.043000000,NaN,36,Current,NaN,0.09065,0.0769,0.0669,0.06546,0.0174,0.04806,7.0,AA,8.0,6,TX,NaN,Other,0.0,True,False,NaN,2014-02-26 04:50:22,820.0,839.0,1972-09-01 00:00:00,6.0,6.0,15.0,6,115.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1201.0,0.03,21789.0,15.0,1.00,0.0,0.11,"$25,000-49,999",True,2916.666667,3E4C3707536403346E613C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,135943,6500,2014-03-07 00:00:00,Q1 2014,580737064463587391804E5,202.76,0.0000,0.00,0.0000,0.00,0.0,0.00,0.00,0.0,1.0,0,0,0.0,119
69760,ED883364810131978EC70C3,31326,2006-08-14 16:45:45.747000000,B,36,Completed,2007-12-26 00:00:00,0.16205,0.1549,0.1449,NaN,NaN,NaN,NaN,NaN,NaN,0,CA,Other,Not available,NaN,False,True,A8113366311579615424760,2006-08-10 08:48:19.837000000,680.0,699.0,1999-08-13 00:00:00,NaN,NaN,20.0,9,413.0,0.0,2.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.38,Not displayed,True,2500.000000,40EE33671767812263FC011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,91,2755,4000,2006-08-22 00:00:00,Q3 2006,6BFD336433924896843BA57,139.62,4562.0000,4000.00,562.0000,-18.75,0.0,0.00,0.00,0.0,1.0,0,0,0.0,69
18481,B3053594310872658DD08B8,993202,2013-11-13 16:22:57.353000000,NaN,36,Current,NaN,0.20984,0.1730,0.1630,0.15264,0.0674,0.08524,4.0,C,4.0,1,CA,Food Service,Employed,228.0,False,False,NaN,2013-11-13 16:23:02,680.0,699.0,2002-02-06 00:00:00,7.0,7.0,13.0,6,437.0,1.0,5.0,0.0,0.0,0.0,1.0,0.0,12980.0,0.64,6866.0,10.0,1.00,0.0,0.31,"$25,000-49,999",True,3750.000000,739436975676133915F5B3D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,4,111697,6000,2013-11-15 00:00:00,Q4 2013,6C6C3698759189006F2E3F0,214.81,641.5862,388.48,253.1062,-14.80,0.0,0.00,0.00,0.0,1.0,0,0,0.0,1
94711,223E3604122041170D63C20,1233672,2014-02-25 14:25:00.100000000,NaN,36,Current,NaN,0.13124,0.1029,0.0929,0.08974,0.

**Issues Identified from Visual Assessment Include:**
- Several missing records in several columns including: `CreditGrade`, `ClosedDate`, `EstimatedEffectiveYield`, `EstimatedLoss`, `EstimatedReturn`, `ProsperRating (numeric)`, `ProsperRating (Alpha)`, `ProsperScore`, `GroupKey`, `TotalProsperLoans`, `TotalProsperPaymentsBilled`, `OnTimeProsperPayments`, `ProsperPaymentsLessThanOneMonthLate`, `ProsperPaymentsOneMonthPlusLate`, `ProsperPrincipalBorrowed`, `ProsperPrincipalOutstanding`, `ScorexChangeAtTimeOfListing`, `LoanFirstDefaultedCycleNumber`
    <br>
- Inconsistent date formats. `ListingCreationDate` column has date format as `yyyy-mm-dd hh:mm:ss.fffffffff` where `ffffffffff` indicates nanoseconds; while `ClosedDate`, `FirstRecordedCreditLine`, and `LoanOriginationDate` columns have date format as `yyyy-mm-dd hh:mm:ss`. `DateCreditPulled` column has records of date in both date formats.
    <br>

- ListingCategory column has numeric values recorded, but from the documentation of [variable definitions](https://docs.google.com/spreadsheets/d/1djKkeentYxKqgNFO2ZHF-KFwVunoZ83VbyLyWYz82M8/edit?usp=sharing), ListingCategory would be better represented in it's categorical form.

*These Issues will be separated into Quality vs Tidiness issues later below in this notebook, after programmatic assessment has been completed*

### Programmatic Assessment

In [16]:
# Obtaining the summary information of the dataset including num of rows and columns, names and datatypes of each column, 
# count of null/non-null values and even memory usage

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 81 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ListingKey                           113937 non-null  object 
 1   ListingNumber                        113937 non-null  int64  
 2   ListingCreationDate                  113937 non-null  object 
 3   CreditGrade                          28953 non-null   object 
 4   Term                                 113937 non-null  int64  
 5   LoanStatus                           113937 non-null  object 
 6   ClosedDate                           55089 non-null   object 
 7   BorrowerAPR                          113912 non-null  float64
 8   BorrowerRate                         113937 non-null  float64
 9   LenderYield                          113937 non-null  float64
 10  EstimatedEffectiveYield              84853 non-null   float64
 11  EstimatedLoss

**Issues Identified So Far**

- Some column names do not follow standard/common naming convention for variables and are not consistent with others; such columns include: ProsperRating (numeric), ProsperRating (Alpha), ListingCategory (numeric), TotalCreditLinespast7years, TradesNeverDelinquent (percentage)
    <br>
    
- Columns containing dates are stored as string/object datatype instead of datetime 

In [9]:
# Displaying the count of unique values in the IncomeRange column
df['IncomeRange'].value_counts()

$25,000-49,999    32192
$50,000-74,999    31050
$100,000+         17337
$75,000-99,999    16916
Not displayed      7741
$1-24,999          7274
Not employed        806
$0                  621
Name: IncomeRange, dtype: int64

**Observation:** Apart from the previously identified tidiness issue, there are entries of 'Not displayed' and 'Not employed' values in the `IncomeRange` column.

In [10]:
# Checking to see if there are any duplicated rows in the dataset
df.duplicated().sum()

0

**Observation:** 

It seems like there are no duplicates. But let's take a closer look at the LoanKey column. 

**Note:** The description of the LoanKey variable is given in the documentation as: Unique key for each loan. This is the same key that is used in the API. *Refer to the complete dictionary of variable definitions [here](https://docs.google.com/spreadsheets/d/1djKkeentYxKqgNFO2ZHF-KFwVunoZ83VbyLyWYz82M8/edit?usp=sharing)*  

Hence there should only be only 1 unique LoanKey for each Loan

In [11]:
# Checking to see if there any duplicates in this column, and getting the count of the duplicates

df['LoanKey'][df['LoanKey'].duplicated()].count()

871

In [12]:
# Checking to get a count of uniques entries in the LoanKey column to ensure we get an accurate number, 
# that when added to the duplicates above, will give the total number of rows in the dataset.

df['LoanKey'].nunique()

113066

In [14]:
# Now adding up the count of uniques entries and the count of the duplicates, 
# and then comparing with the length of the dataframe (i.e total num of rows in dataset). Should return "True" 

df['LoanKey'].nunique() + df['LoanKey'][df['LoanKey'].duplicated()].count() == len(df)

True

In [15]:
# Displaying the original and duplicated entries in the LoanKey column, to investigate if they are truly duplicates.
df[df['LoanKey'].duplicated(keep=False)].sort_values('LoanKey')

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,EstimatedEffectiveYield,EstimatedLoss,EstimatedReturn,ProsperRating (numeric),ProsperRating (Alpha),ProsperScore,ListingCategory (numeric),BorrowerState,Occupation,EmploymentStatus,EmploymentStatusDuration,IsBorrowerHomeowner,CurrentlyInGroup,GroupKey,DateCreditPulled,CreditScoreRangeLower,CreditScoreRangeUpper,FirstRecordedCreditLine,CurrentCreditLines,OpenCreditLines,TotalCreditLinespast7years,OpenRevolvingAccounts,OpenRevolvingMonthlyPayment,InquiriesLast6Months,TotalInquiries,CurrentDelinquencies,AmountDelinquent,DelinquenciesLast7Years,PublicRecordsLast10Years,PublicRecordsLast12Months,RevolvingCreditBalance,BankcardUtilization,AvailableBankcardCredit,TotalTrades,TradesNeverDelinquent (percentage),TradesOpenedLast6Months,DebtToIncomeRatio,IncomeRange,IncomeVerifiable,StatedMonthlyIncome,LoanKey,TotalProsperLoans,TotalProsperPaymentsBilled,OnTimeProsperPayments,ProsperPaymentsLessThanOneMonthLate,ProsperPaymentsOneMonthPlusLate,ProsperPrincipalBorrowed,ProsperPrincipalOutstanding,ScorexChangeAtTimeOfListing,LoanCurrentDaysDelinquent,LoanFirstDefaultedCycleNumber,LoanMonthsSinceOrigination,LoanNumber,LoanOriginalAmount,LoanOriginationDate,LoanOriginationQuarter,MemberKey,MonthlyLoanPayment,LP_CustomerPayments,LP_CustomerPrincipalPayments,LP_InterestandFees,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
55674,AB5D3591856313013078E5F,953464,2013-10-21 17:07:52.680000000,NaN,60,Current,NaN,0.15016,0.1274,0.1174,0.11210,0.0424,0.06970,5.0,B,9.0,1,OH,Other,Employed,11.0,True,False,NaN,2013-12-15 22:35:50,680.0,699.0,1984-03-01 00:00:00,12.0,12.0,32.0,10,920.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,17219.0,0.36,20033.0,22.0,0.86,2.0,0.30,"$75,000-99,999",True,6250.000000,001D370202065948445765E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,3,122232,20000,2013-12-23 00:00:00,Q4 2013,ED1E3696011133656285632,452.40,904.8000,474.54,430.2600,-33.78,0.0,0.0,0.0,0.0,1.0,0,0,0.0,291
54740,AB5D3591856313013078E5F,953464,2013-10-21 17:07:52.680000000,NaN,60,Current,NaN,0.15016,0.1274,0.1174,0.11210,0.0424,0.06970,5.0,B,8.0,1,OH,Other,Employed,11.0,True,False,NaN,2013-12-15 22:35:50,680.0,699.0,1984-03-01 00:00:00,12.0,12.0,32.0,10,920.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,17219.0,0.36,20033.0,22.0,0.86,2.0,0.30,"$75,000-99,999",True,6250.000000,001D370202065948445765E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,3,122232,20000,2013-12-23 00:00:00,Q4 2013,ED1E3696011133656285632,452.40,904.8000,474.54,430.2600,-33.78,0.0,0.0,0.0,0.0,1.0,0,0,0.0,291
87726,B08A359338973786631D76F,959341,2013-10-25 10:50:48.623000000,NaN,36,Current,NaN,0.13301,0.1050,0.0950,0.09202,0.0274,0.06462,6.0,A,6.0,1,AL,NaN,Other,37.0,True,False,NaN,2013-10-10 13:58:15,740.0,759.0,1998-03-14 00:00:00,17.0,16.0,37.0,12,378.0,1.0,5.0,0.0,0.0,0.0,1.0,0.0,8563.0,0.21,30847.0,36.0,1.00,1.0,0.30,"$25,000-49,999",True,3000.000000,009C36959125842600757B0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,5,107589,9000,2013-10-29 00:00:00,Q4 2013,35F9369605565425820B0D9,292.52,1167.4910,865.44,302.0510,-29.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
21442,B08A359338973786631D76F,959341,2013-10-25 10:50:48.623000000,NaN,36,Current,NaN,0.13301,0.1050,0.0950,0.09202,0.0274,0.06462,6.0,A,7.0,1,AL,NaN,Other,37.0,True,False,NaN,2013-10-10 13:58:15,740.0,759.0,1998-03-14 00:00:00,17.0,16.0,37.0,12,378.0,1.0,5.0,0.0,0.0,0.0,1.0,0.0,8563.0,0.21,30847.0,36.0,1.00,1.0,0.30,"$25,000-49,999",True,3000.000000,009C36959125842600757B0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,5,107589,9000,2013-10-29 00:00:00,Q4 2013,35F9369605565425820B0D9,292.52,1167.4910,865.44,302.0510,-29.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
43779,33A83600436066363252CED,1158389,2014-01-24 11:42:23.370000000,NaN,36,Current,NaN,0.12691,0.0990,0.0890,0.08619,0.0274,0.05879,6.0,A,5.0,1,GA,Computer Programmer,Employed,

**Observation:**

The result above confirms that the entries are truly duplicates, as the only difference between the original and duplicate entry is the 'ProsperScore' variable. Now let's see the [data dictionary](https://docs.google.com/spreadsheets/d/1djKkeentYxKqgNFO2ZHF-KFwVunoZ83VbyLyWYz82M8/edit?usp=sharing) for the definition of the 'ProsperScore' variable.

_**ProsperScore** - A custom risk score built using historical Prosper data. The score ranges from 1-10, with 10 being the best, or lowest risk score.  Applicable for loans originated after July 2009._

From the data dictionary definition of the prosper score, it is safe to assume that for all duplicate entries in the dataset, the first entry is the original and the second is most likely the duplicate. 

**Issues Identified So Far**
- Some column names do not follow standard/common naming convention for variables and are not consistent with others; such columns include: ProsperRating (numeric), ProsperRating (Alpha), ListingCategory (numeric), TotalCreditLinespast7years, TradesNeverDelinquent (percentage)
    <br>
    
- Columns containing dates are stored as string/object datatype instead of datetime
    <br>
    
- Duplicate entries in the dataset

<br>

In [14]:
# Obtaining the descriptive statistics for the numeric variables in the dataset to see if there are strange entries or outliers based on the column definition
df.describe()

,ListingNumber,Term,BorrowerAPR,BorrowerRate,LenderYield,EstimatedEffectiveYield,EstimatedLoss,EstimatedReturn,ProsperRating (numeric),ProsperScore,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
count,1.139370e+05,113937.000000,113912.000000,113937.000000,113937.000000,84853.000000,84853.000000,84853.000000,84853.000000,84853.000000,...,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000
mean,6.278857e+05,40.830248,0.218828,0.192764,0.182701,0.168661,0.080306,0.096068,4.072243,5.950067,...,-54.725641,-14.242698,700.446342,681.420499,25.142686,0.998584,0.048027,0.023460,16.550751,80.475228
std,3.280762e+05,10.436212,0.080364,0.074818,0.074516,0.068467,0.046764,0.030403,1.673227,2.376501,...,60.675425,109.232758,2388.513831,2357.167068,275.657937,0.017919,0.332353,0.232412,294.545422,103.239020
min,4.000000e+00,12.000000,0.006530,0.000000,-0.010000,-0.182700,0.004900,-0.182700,1.000000,1.000000,...,-664.870000,-9274.750000,-94.200000,-954.550000,0.000000,0.700000,0.000000,0.000000,0.000000,1.000000
25%,4.009190e+05,36.000000,0.156290,0.134000,0.124200,0.115670,0.042400,0.074080,3.000000,4.000000,...,-73.180000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000
50%,6.005540e+05,36.000000,0.209760,0.184000,0.173000,0.161500,0.072400,0.091700,4.000000,6.000000,...,-34.440000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,44.000000
75%,8.926340e+05,36.000000,0.283810,0.250000,0.240000,0.224300,0.112000,0.116600,5.000000,8.000000,...,-13.920000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,115.000000
max,1.255725e+06,60.000000,0.512290,0.497500,0.492500,0.319900,0.366000,0.283700,7.000000,11.000000,...,32.060000,0.000000,25000.000000,25000.000000,21117.900000,1.012500,39.000000,33.000000,25000.000000,1189.000000


**Observation:**

No strange entries or immediate outliers detected from the descriptive statistics result shown above

In [25]:
# Now I want to find out how many missing values are in each variable that we previously detected during visual assessment
df.isna().sum()

ListingKey                                  0
ListingNumber                               0
ListingCreationDate                         0
CreditGrade                             84984
Term                                        0
LoanStatus                                  0
ClosedDate                              58848
BorrowerAPR                                25
BorrowerRate                                0
LenderYield                                 0
EstimatedEffectiveYield                 29084
EstimatedLoss                           29084
EstimatedReturn                         29084
ProsperRating (numeric)                 29084
ProsperRating (Alpha)                   29084
ProsperScore                            29084
ListingCategory (numeric)                   0
BorrowerState                            5515
Occupation                               3588
EmploymentStatus                         2255
EmploymentStatusDuration                 7625
IsBorrowerHomeowner               

In [16]:
# Investigating the reason why there are missing values; starting with the CreditGrade column
df['CreditGrade']

0           C
1         NaN
2          HR
3         NaN
4         NaN
         ... 
113932    NaN
113933    NaN
113934    NaN
113935    NaN
113936    NaN
Name: CreditGrade, Length: 113937, dtype: object

In [27]:
# The values in the CreditGrade variable above look similar to values of the ProsperRating (Alpha) variable that I've seen before
# So I'll display just these two variables to investigate further

df[['CreditGrade', 'ProsperRating (Alpha)']]

,CreditGrade,ProsperRating (Alpha)
0,C,NaN
1,NaN,A
2,HR,NaN
3,NaN,A
4,NaN,D
...,...,...
113932,NaN,C
113933,NaN,A
113934,NaN,D
113935,NaN,C


**Observation:**

They both contain the same values, so I'll refer to the data dictionary of variable definitions [here](https://docs.google.com/spreadsheets/d/1djKkeentYxKqgNFO2ZHF-KFwVunoZ83VbyLyWYz82M8/edit?usp=sharing) to understand why.

**`CreditGrade`** was defined as the Credit rating that was assigned at the time the listing went live. Applicable for listings pre-2009 period and will only be populated for those listings.

**`ProsperRating (Alpha)`** was defined as the Prosper Rating (between AA - HR) assigned at the time the listing was created.  Applicable for loans originated after July 2009.

*Hence both variables should not be in separate columns, which explains why there are so many missing values in CreditGrade, ProsperRating (Alpha) and ProsperRating (numeric) columns. This wil be added to the list of quality and tidiness issues to be resolved*

<br>


In [368]:
# Checking the different status of loans in the dataset
df['LoanStatus'].value_counts()

Current                   56576
Completed                 38074
Chargedoff                11992
Defaulted                  5018
Past Due (1-15 days)        806
Past Due (31-60 days)       363
Past Due (61-90 days)       313
Past Due (91-120 days)      304
Past Due (16-30 days)       265
FinalPaymentInProgress      205
Past Due (>120 days)         16
Cancelled                     5
Name: LoanStatus, dtype: int64

**Observation:** 

The variable definition dictionary did not properly define what each loan status means. So I found an online dictionary to define what each loan status terminology means.. Check out [LawInsider's Dictionary](https://www.lawinsider.com/dictionary/chargedoff-loan). It explains the different terminologies found in the LoanStatus variable/column.

_**P.S:** I will be using `LawInsider's API` to provide access to their dictionary for users who want to search the meaning of loan status terms directly from this notebook. Come back for a future release version of this notebook_

<BR>

**List of Identified Issues from Programmatic Assessment**
- Some column names do not follow standard/common naming convention for variables and are not consistent with others; such columns include: `ProsperRating (numeric)`, `ProsperRating (Alpha)`, `ListingCategory (numeric)`, `TotalCreditLinespast7years`, `TradesNeverDelinquent (percentage)`

    
- Columns containing dates are stored as string/object datatype instead of datetime
    <br>
    
- Duplicate entries in the dataset


- Multiple columns for the same variable. `CreditGrade` and `ProsperRating (Alpha)` data should not be in 2 separate columns

For the other columns with missing values, the documentation explains why these values are null.
For example, the [data dictionary](https://docs.google.com/spreadsheets/d/1djKkeentYxKqgNFO2ZHF-KFwVunoZ83VbyLyWYz82M8/edit?usp=sharing) explains that only cancelled, completed, charged off or defauled loans have values in the `ClosedDate` column.

It also explains that only current loans originated after July 2009 have values in the `EstimatedEffectiveYield`, `EstimatedLoss`, `EstimatedReturn`, `ProsperRating (numeric)`, `ProsperRating (Alpha)`, and `ProsperScore` columns. That means completed, cancelled, charged off, defaulted or past due loans will have missing values in these columns.

The same applies for the other columns with missing values in the dataset. 

   <br>

_Refer to the data dictionary [here](https://docs.google.com/spreadsheets/d/1djKkeentYxKqgNFO2ZHF-KFwVunoZ83VbyLyWYz82M8/edit?usp=sharing) to understand the values in each variable._

#### All Tidiness Issues

- Multiple columns for the same variable. `CreditGrade` and `ProsperRating (Alpha)` data should not be in 2 separate columns


#### All Quality Issues
- Columns containing dates are stored as string/object datatype instead of datetime, and inconsistent date formats. `ListingCreationDate` column has date format as `yyyy-mm-dd hh:mm:ss.fffffffff` where `ffffffffff` indicates nanoseconds; while `ClosedDate`, `FirstRecordedCreditLine`, and `LoanOriginationDate` columns have date format as `yyyy-mm-dd hh:mm:ss`. `DateCreditPulled` column has records of date in both date formats.


- `ListingCategory (numereic)` column has numeric values recorded, but from the documentation of variable definitions, Listing Category would be better represented in it's categorical form.


- Some column names do not follow standard/common naming convention for variables and are not consistent with others; such columns include: `ProsperRating (numeric)`, `ProsperRating (Alpha)`, `ListingCategory (numeric)`, `TotalCreditLinespast7years`, `TradesNeverDelinquent (percentage)`


- Duplicate entries in the dataset


## Cleaning the Data

In this section, all tidiness and quality issues are cleaned using the define-code-test framework which involves defining cleaning steps using verbs and action words that clearly describe the cleaning tasks,
performing the cleaning tasks accordingly and testing programmatically to see if my desired results were
achieved.

In [72]:
# First things first, making a copy of the original dataset so that if there any issues, I have my original file intact.
data = df.copy()

### Tidiness Issue 1

#### Define

Consolidate values in `CreditGrade` and `ProsperRating (Alpha)` columns into one (1) column


#### Code

In [73]:
# First, replace all null values with empty string in CreditGrade colum
data['CreditGrade'].replace(np.NaN, '', inplace=True)

# Doing the same for the ProsperRating (Alpha) column, replacing all null values with empty string
data['ProsperRating (Alpha)'].replace(np.NaN, '', inplace=True)

# Now create a new column 'ProsperRating' that combines all values in CreditGrade and ProsperRating (Alpha) columns
data['ProsperRating'] = data['CreditGrade'] + data['ProsperRating (Alpha)']

# Remove newly created 'ProsperRating' column from the dataframe and assign to a variable that will be reinserted at the index location of the 'CreditGrade' column
rating_col = data.pop('ProsperRating')

# Get column index location of 'CreditGrade' column and used that index location to insert the new 'ProsperRating' column back into the dataframe
data.insert(loc=data.columns.get_loc('CreditGrade'), column='ProsperRating', value=rating_col)

# Finally, remove both CreditGrade and ProsperRating (Alpha) columns
data.drop(['CreditGrade', 'ProsperRating (Alpha)'], axis=1, inplace=True)

#### Test

In [74]:
# Displaying the count of unique values in the newly created 'CreditRating' column
data['ProsperRating'].value_counts()

ProsperRating
C     23994
B     19970
D     19427
A     17866
E     13084
HR    10443
AA     8881
NC      141
        131
Name: count, dtype: int64

In [75]:
# Checking to make sure the previous columns CreditGrade and ProsperRating (Alpha), have been removed
('CreditGrade', 'ProsperRating (Alpha)') in data.columns

False

### Quality Issue 1

#### Define

Change datatype in columns containing dates from string/object to datetime datatype. Ensure date records with timestamp data are in `yyyy-mm-dd hh:mm:ss` format where applicable. 

Date columns are:
- `ListingCreationDate`
- `ClosedDate`
- `FirstRecordedCreditLine`
- `LoanOriginationDate`
- `DateCreditPulled`

#### Code

In [76]:
# Creating a list of columns that have values of dates i.e date columns
date_columns = ['ListingCreationDate', 'ClosedDate', 'FirstRecordedCreditLine', 'LoanOriginationDate', 'DateCreditPulled']

# Using a for loop to convert all the date columns from string/object to datetime data type.
for column in date_columns:
    data[column] = pd.to_datetime(data[column].str.split(".").str[0])

#### Test

In [77]:
# Displaying the first 5 rows of the now converted date columns to check if cleaning task was successful
data[['ListingCreationDate', 'ClosedDate', 'FirstRecordedCreditLine', 'LoanOriginationDate', 'DateCreditPulled']].head()

,ListingCreationDate,ClosedDate,FirstRecordedCreditLine,LoanOriginationDate,DateCreditPulled
0,2007-08-26 19:09:29,2009-08-14,2001-10-11,2007-09-12,2007-08-26 18:41:46
1,2014-02-27 08:28:07,NaT,1996-03-18,2014-03-03,2014-02-27 08:28:14
2,2007-01-05 15:00:47,2009-12-17,2002-07-27,2007-01-17,2007-01-02 14:09:10
3,2012-10-22 11:02:35,NaT,1983-02-28,2012-11-01,2012-10-22 11:02:32
4,2013-09-14 18:38:39,NaT,2004-02-20,2013-09-20,2013-09-14 18:38:44


### Quality Issue 2

#### Define
Replace numeric values in `ListingCategory (numeric)` column with it's corresponding categorical data, and then rename the column to `ListingCategory`

#### Code

In [78]:
# Create a dictionary using listing category data provided in documentation of variable definitions, to map each numeric value to it's corresponding categorical data
categories = {0: 'Not Available', 1: 'Debt Consolidation', 2: 'Home Improvement', 3: 'Business', 4: 'Personal Loan', 5: 'Student Use', 6: 'Auto', 7: 'Other', 8: 'Baby And Adoption', 9: 'Boat', 10: 'Cosmetic Procedure', 11: 'Engagement Ring', 12: 'Green Loans', 13: 'Household Expenses', 14: 'Large Purchases', 15: 'Medical/Dental', 16: 'Motorcycle', 17: 'RV', 18: 'Taxes', 19: 'Vacation', 20: 'Wedding Loans'}

# Replace the numeric values in ListingCategory (numeric) column with it's corresponding categorical data
data.replace({'ListingCategory (numeric)': categories}, inplace=True)

# Rename the column to just 'ListingCategory'
data.rename(columns={'ListingCategory (numeric)': 'ListingCategory'}, inplace=True)

#### Test

In [79]:
# Displaying the count of unique values to ensure all numeric values were replaced with their corresponding categorical data
data['ListingCategory'].value_counts()

ListingCategory
Debt Consolidation    58308
Not Available         16965
Other                 10494
Home Improvement       7433
Business               7189
Auto                   2572
Personal Loan          2395
Household Expenses     1996
Medical/Dental         1522
Taxes                   885
Large Purchases         876
Wedding Loans           771
Vacation                768
Student Use             756
Motorcycle              304
Engagement Ring         217
Baby And Adoption       199
Cosmetic Procedure       91
Boat                     85
Green Loans              59
RV                       52
Name: count, dtype: int64

### Quality Issue 3

#### Define

Rename columns with names and characters that follow consistent column naming convention already being used in the dataset.

**Note:** *Since 'ListingCategory (numeric)' has already been renamed above, it was not included in this section. Also 'ProsperRating (Alpha)' has already been removed from the dataset after it's values were combined with 'CreditGrade' earlier to form the new `CreditRating` column* 

Columns to be renamed are: 
- `ProsperRating (numeric)`  
- `TotalCreditLinespast7years` 
- `TradesNeverDelinquent (percentage)`

#### Code

In [80]:
# Renaming the listed columns above to be consistent with other column names
data.rename(columns={"ProsperRating (numeric)": "ProsperRatingNumeric", "TotalCreditLinespast7years": "TotalCreditLinesPast7Years", "TradesNeverDelinquent (percentage)": "PercentTradesNeverDelinquent"}, inplace=True)

#### Test

In [81]:
# Checking to make sure the previous columns ProsperRating (numeric), TradesNeverDelinquent (percentage), and TradesNeverDelinquent (percentage), have been removed
('ProsperRating (numeric)', 'TradesNeverDelinquent (percentage)', 'TradesNeverDelinquent (percentage)') in data.columns

False

In [82]:
# Displaying current column names
print(data.columns)

Index(['ListingKey', 'ListingNumber', 'ListingCreationDate', 'ProsperRating',
       'Term', 'LoanStatus', 'ClosedDate', 'BorrowerAPR', 'BorrowerRate',
       'LenderYield', 'EstimatedEffectiveYield', 'EstimatedLoss',
       'EstimatedReturn', 'ProsperRatingNumeric', 'ProsperScore',
       'ListingCategory', 'BorrowerState', 'Occupation', 'EmploymentStatus',
       'EmploymentStatusDuration', 'IsBorrowerHomeowner', 'CurrentlyInGroup',
       'GroupKey', 'DateCreditPulled', 'CreditScoreRangeLower',
       'CreditScoreRangeUpper', 'FirstRecordedCreditLine',
       'CurrentCreditLines', 'OpenCreditLines', 'TotalCreditLinesPast7Years',
       'OpenRevolvingAccounts', 'OpenRevolvingMonthlyPayment',
       'InquiriesLast6Months', 'TotalInquiries', 'CurrentDelinquencies',
       'AmountDelinquent', 'DelinquenciesLast7Years',
       'PublicRecordsLast10Years', 'PublicRecordsLast12Months',
       'RevolvingCreditBalance', 'BankcardUtilization',
       'AvailableBankcardCredit', 'TotalTrades',
 

### Quality Issue 4

#### Define

Duplicate entries in the dataset. Keep the first entry and remove the second entry which is the duplicate.

#### Code



In [83]:
# Remove the duplicate entries while keeping the first occurrence in the df 
df = df.drop_duplicates(subset='LoanKey', keep='first').sort_values('LoanKey')

#### Test

In [84]:
# Checking to get the count of the duplicates left if any
df['LoanKey'][df['LoanKey'].duplicated()].count()

0

## Exploratory Data Analysis (EDA)

In this section, exploratory data analysis (EDA) is used to understand the patterns and relationships between variables in the dataset. This understanding is useful to:
- approach any statistical analysis in order to generate findings 
- draw conclusions and make recommendations. 

_This process might also reveal additional data cleaning tasks that need to be done to get the data ready for statistical modeling or predictive analyses._

<br>

**Why is EDA important?**

Exploratory Data Analysis of Prosper loan data can provide insights into a variety of areas, including:
- Credit risk
- Borrowing patterns
- Trends in personal loan finance. 

For example, by analyzing the credit scores and income levels of borrowers, we can gain insight into the types of individuals who are most likely to default on loans. Additionally, the data can be used to identify patterns in borrowing behavior, such as the types of loans that are most popular among different demographics or the average loan amounts for different types of borrowers. 

Other insights that can be gotten include:
- the geographic distribution of borrowers
- the length of time it takes for loans to be repaid
- the overall performance of the loan portfolio

<br>


### Univariate EDA

From data wrangling, the variables of top interest include:
- `ProsperRating`: This variable represents the credit rating assigned to the borrower by Prosper. It can provide insights into the creditworthiness of borrowers.
- `LoanStatus`: indicates the current status of the loan, which can be useful for analyzing loan performance and default rates.
- `BorrowerAPR`: represents the Annual Percentage Rate (APR) assigned to the borrower, providing insights into the cost of borrowing.
- `LoanOriginalAmount`: indicates the original loan amount, which can be relevant for understanding loan sizes and their impact on various outcomes.
- `ListingCategory`: represents the category of the loan listing. It can provide insights into the purpose or intended use of the loan.
- `EmploymentStatus`: captures the employment status of the borrower, offering insights into the relationship between employment and loan performance.
- `IncomeRange`: This variable represents the income range of the borrower. It can provide insights into the income levels of borrowers and their ability to repay loans.
- `StatedMonthlyIncome`: This variable represents the borrower's stated monthly income. It can provide insights into income levels and affordability.

... just to name the vital few

<br>
